In [1]:
import cv2
import dlib
import numpy as np
from keras.models import load_model

# Load pre-trained models
face_detector = dlib.get_frontal_face_detector()
landmark_detector = dlib.shape_predictor("DATA/shape_predictor_68_face_landmarks.dat")
emotion_classifier = load_model("DATA/EmotionDetectionModel.h5")

# Define emotion labels
EMOTIONS = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Initialize video capture
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the video stream
    ret, frame = cap.read()

    # Detect faces in the frame
    faces = face_detector(frame, 0)

    # Iterate over detected faces
    for face in faces:
        # Align face using facial landmarks
        landmarks = landmark_detector(frame, face)
        aligned_face = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(0, 68)])

        # Preprocess aligned face for emotion classification
        grayscale_face = cv2.cvtColor(aligned_face, cv2.COLOR_BGR2GRAY)
        resized_face = cv2.resize(grayscale_face, (48, 48))
        reshaped_face = resized_face.reshape(1, 48, 48, 1)

        # Predict emotions for aligned face
        predictions = emotion_classifier.predict(reshaped_face)
        emotion_label = EMOTIONS[np.argmax(predictions)]

        # Draw bounding box and emotion label on face in video stream
        (x, y, w, h) = face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion_label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the resulting video stream
    cv2.imshow('Emotion Detection', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'cv2'